<a href="https://colab.research.google.com/github/RainaVardhan/project_voting/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! git clone https://github.com/DS3001/project_voting.git

Cloning into 'project_voting'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 70 (delta 5), reused 3 (delta 3), pack-reused 62
Receiving objects: 100% (70/70), 31.37 MiB | 11.69 MiB/s, done.
Resolving deltas: 100% (29/29), done.


In [144]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/project_voting/data/county_data/0002_ds250_20205_county_M.csv', encoding='ISO-8859-1') #asked chat how to fix encoding error
df['U.S. citizen: Born in the US'] = df['AM0KM002']
df['U.S. citizen: Born abroad'] = df['AM0KM004']
df['U.S. citizen: Naturalized'] = df['AM0KM005']
df['Male'] = df['AM8FM002']
df['Female'] = df['AM8FM021']
df['18 to 30 years'] = df['AM8FM009']+df['AM8FM028']
df['35 to 64 years'] = df['AM8FM012']+df['AM8FM031']
df['65 to 74 years'] = df['AM8FM015']+df['AM8FM034']
df['75 years and over'] = df['AM8FM018']+df['AM8FM037']

virginia_county_data = df[df['STATE'] == 'Virginia']

vars_to_keep = ['COUNTY','U.S. citizen: Born in the US', 'U.S. citizen: Born abroad', 'U.S. citizen: Naturalized', 'Male', 'Female', '18 to 30 years', '35 to 64 years', '65 to 74 years', '75 years and over']
county_data = virginia_county_data[vars_to_keep]
county_data = county_data.drop(virginia_county_data.index[0])
county_data.head()

county_data.isna().sum()
county_data = county_data.dropna()
county_data.isna().sum()
print(county_data.head())


                 COUNTY U.S. citizen: Born in the US  \
2823   Albemarle County                          936   
2824   Alleghany County                          121   
2825      Amelia County                          142   
2826     Amherst County                          190   
2827  Appomattox County                           89   

     U.S. citizen: Born abroad U.S. citizen: Naturalized Male Female  \
2823                       299                       626  193    166   
2824                        96                        45  137    112   
2825                       122                       105  131    135   
2826                        77                       132  154    169   
2827                        39                        51   87     90   

     18 to 30 years 35 to 64 years 65 to 74 years 75 years and over  
2823          16785          12690           9079             80159  
2824           6241           8473           5165              3160  
2825          14879 

In [ ]:
df = pd.read_csv('/content/project_voting/data/voting_VA.csv')
#df.head()

# check for nan values
print(df.isnull().values.any())

# check for possible mistakes
df[df['totalvotes'] == 0]

# split the dataframe into each year
votes_2012 = df[df['year']==2012]
votes_2016 = df[df['year']==2016]
votes_2020 = df[df['year']==2020]

# find out which party got the most votes in each county
winners_2012_idx = votes_2012.groupby('county_name')['candidatevotes'].idxmax() # find maximum number of candidatevotes for county using idxmax
winners_2012 = votes_2012.loc[winners_2012_idx]
winners_2016_idx = votes_2016.groupby('county_name')['candidatevotes'].idxmax()
winners_2016 = votes_2016.loc[winners_2016_idx]
winners_2020_idx = votes_2020.groupby('county_name')['candidatevotes'].idxmax()
winners_2020 = votes_2020.loc[winners_2020_idx]

# combine the dataframes into 1
winners_all = pd.concat([winners_2012,winners_2016,winners_2020])

winners_all

votes_2016[votes_2016['county_name'] == 'BEDFORD']
# the 0 values seems to be an accidental double entry = don't have to clean

In [168]:
winners_2020.head()

,Unnamed: 0,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
2154,70389,2020,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,6078,16962,20220315,ELECTION DAY
2156,70391,2020,VIRGINIA,VA,ALBEMARLE,51003,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,33416,64657,20220315,ABSENTEE
3284,71519,2020,VIRGINIA,VA,ALEXANDRIA CITY,51510,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,55940,82521,20220315,ABSENTEE
2178,70413,2020,VIRGINIA,VA,ALLEGHANY,51005,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,4409,8203,20220315,ELECTION DAY
2190,70425,2020,VIRGINIA,VA,AMELIA,51007,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,2778,7894,20220315,ELECTION DAY


In [164]:
winners_2020.shape

(133, 13)

In [178]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score

# county_data['COUNTY'] = county_data['COUNTY'].str.replace('county', '', case=False).str.capitalize()

votes = winners_2020.groupby('county_name').agg({'candidatevotes':'sum'}).reset_index()
print(votes)
county_data['COUNTY'] = county_data['COUNTY'].str.upper()
votes['county_name'] = votes['county_name'].str.upper()
combined = pd.merge(county_data,votes,left_on='COUNTY', right_on='county_name', how='inner')
print(combined.head())
# merged_data = pd.merge(county_data, winners_2020, left_on='COUNTY', right_on='county_name')

# print(merged_data.head())

# print(county_data.head())
# county_values = county_data['COUNTY'].unique()
# print(county_values)

# # Define selected features and target variable
# selected_features = ['U.S. citizen: Born in the US', '18 to 30 years', '35 to 64 years', '65 to 74 years', '75 years and over']
# target_variable = 'COUNTY'

# # Filter dataset to include only selected features and target variable
# trainData = county_data[selected_features + [target_variable]]

# # Drop rows with missing values
# trainData.dropna(inplace=True)

# # Split training and testing data into features (X) and target variable (y)
# X = trainData[selected_features]
# y = trainData[target_variable]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# # Train Decision Tree model
# clf = DecisionTreeClassifier(max_depth=10, min_samples_leaf=5, random_state=42)
# clf.fit(X_train, y_train)

# # Visualize the decision tree
# plt.figure(figsize=(20, 25))
# plot_tree(clf, filled=True, feature_names=selected_features, impurity=False)
# plt.show()

# # Predict on test set
# y_pred = clf.predict(X_test)

# # Evaluate model performance
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)

        party  candidatevotes
0    DEMOCRAT         1547152
1  REPUBLICAN          484581


KeyError: 'county_name'